# 卷积操作理解

从线性代数视角看，二维卷积是在输入特征图上滑动卷积核，对局部 patch 做加权求和并写回输出。每个输出元素对应一个感受野的响应，卷积核的权值模式决定它对哪类局部结构敏感；共享同一 kernel 遍历整张图，减少参数且符合 DRY。

## 手工计算示意

假设输入特征图 $X$ 和卷积核 $K$ 如下（stride=1, padding=0）：

```
X (5×5)                       K (3×3)
[1  2  0  3  1]               [ 1  0 -1]
[4  5  1  0  2]               [ 1  0 -1]
[0  2  3  2  0]               [ 1  0 -1]
[1  1  0  1  3]
[2  0  1  2  2]
```

卷积核左上角对齐 $(0,0)$ 时：

$$Y_{0,0} = \sum_{u=0}^{2} \sum_{v=0}^{2} X_{u,v} \cdot K_{u,v} = 1 + 4 - 1 - 3 = 1$$

沿横纵方向滑动即可得到尺寸为 $3\times3$ 的输出特征图。扩展到多通道时，卷积核形状为 $C_{\text{out}} \times C_{\text{in}} \times k_H \times k_W$，对每个输入通道的结果求和即可获得对应输出通道像素。

In [ ]:
import torch  # 导入 PyTorch

input_tensor = torch.tensor([  # 构造输入矩阵
    [1, 2, 0, 3, 1],
    [4, 5, 1, 0, 2],
    [0, 2, 3, 2, 0],
    [1, 1, 0, 1, 3],
    [2, 0, 1, 2, 2]
], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # 形状变为 (1,1,5,5) 以匹配 Conv2d

kernel = torch.tensor([  # 构造卷积核
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # 形状为 (1,1,3,3)

  # 因此 (1, 1, 3, 3) 可以直译为“生成 1 个输出通道、输入是 1 个通道、核大小为 3×3 的卷积层权值”。
  # 如果是 (16, 3, 5, 5)，则表示：输入是 3 通道，输出 16 个特征图，每个卷积核大小 5×5。对于 groups
  # 参数不为 1 的情况，形状含义会细化为 (out_channels, in_channels / groups, kH, kW)，常用于深度可分离卷积或分组卷积



# • 在引入 padding 后，卷积核张量本身的形状仍然是 (out_channels, in_channels, kernel_height, kernel_width)，
#   不发生改变；变化的是输入张量在卷积前被“扩边”后的有效尺寸，从而影响感受野覆盖范围与输出特征图大小。
#
  #当输入引入padding时，输出尺寸会发生变化(也就是输出特征图的大小会发生变化:多一圈 )
  #从而使得输出特征图的大小与输入特征图的大小相同,也就是让边缘的像素也能够被卷积核覆盖到
  #  - 以单通道输入 H×W、卷积核 kH×kW、步长 stride=s 为例，输出尺寸变为
  #     H_out = ⌊(H + 2p - kH) / s⌋ + 1，
  #     W_out = ⌊(W + 2p - kW) / s⌋ + 1。



# • - 卷积核与通道的对应关系：(in_channels 必须与输入张量的通道数严格一致)
# Conv2d(in_channels=C_in, out_channels=C_out, kernel_size=kH×kW)
# 会创建 C_out 组权重；每组内含 C_in 个 kH×kW 小矩阵。也就是说，输出通道数直接由 out_channels 决定：
#  有 2 个输出通道，就有 2 组完整的权重堆叠 : 也就是有两个卷积核
#  每组会对全部输入通道做匹配计算，再把这些通道贡献求和，形成该输出通道在某个空间位置的一次响应。

 # kernel_size 是卷积核的大小, 也就是 kH×kW
conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)  # 创建卷积层
with torch.no_grad():  # 关闭梯度便于设定权重
    conv.weight.copy_(kernel)  # 将自定义 kernel 复制到卷积层


output = conv(input_tensor)  # 执行卷积
print(output.squeeze())  # 打印 (3,3) 输出矩阵，核对手算结果





## 结果解析

输出矩阵中的每个值表征对应感受野与卷积核模式的匹配程度。卷积核通过训练得到，能够自动聚焦特定纹理或边缘，从而在深层网络中逐级构建复杂特征。

# 卷积的logits理解 :

In [ ]:
  # 输入图像 (3×32×32)
  #     ↓
  # [特征提取阶段] - 学习"是什么样"
  #     ↓ 卷积1 → 提取边缘、纹理 (32个特征图)
  #     ↓ 卷积2 → 组合成角点、简单形状 (32个特征图)
  #     ↓ 卷积3 → 抽象成物体部件 (64个特征图)
  #     ↓
  # 特征向量 (1024维) ← "图像的数字指纹"
  #     ↓
  # [降维阶段] - 提炼关键信息
  #     ↓ 全连接1 → 压缩到64维语义特征
  #     ↓
  # 64维语义向量 ← "图像的精华表示"
  #     ↓
  # [分类阶段] - 判断"是哪一类"
  #     ↓ 全连接2 → 与10个类别模板匹配
  #     ↓
  # 10个 logits ← "对10个类别的原始评分"
  #
  # 2. 关键理解:最后一层的数学
  #
  # # 最后一层: Linear(64, 10)
  # #
  # # 权重形状: (10, 64)
  # #   ┌─────────── 64个输入特征 ──────────┐
  # #   │                                    │
  # # ┌─┤  w₀₀  w₀₁  w₀₂  ...  w₀₆₃        │ ← 类别0(飞机)的模板
  # # │ │  w₁₀  w₁₁  w₁₂  ...  w₁₆₃        │ ← 类别1(汽车)的模板
  # # │ │  w₂₀  w₂₁  w₂₂  ...  w₂₆₃        │ ← 类别2(鸟)的模板
  # # 10│   ⋮    ⋮    ⋮    ⋱    ⋮          │    ⋮
  # # 类│  w₉₀  w₉₁  w₉₂  ...  w₉₆₃        │ ← 类别9(卡车)的模板
  # # │ └────────────────────────────────────┘
  # # └─ 每一行是一个"类别模板"
  #
  # # 计算 logits:
  # # logit[i] = 特征向量 · 类别i的模板 + 偏置[i]
  # #          = Σ(feature[j] × weight[i,j]) + bias[i]
  # #          = "特征与模板的相似度"
  #
  # 3. 直观类比
  #
  # 想象你在识别动物:
  #
  # # 64维特征向量就像"特征清单":
  # features = [
  #     有翅膀:0.8,    # 特征0
  #     有轮子:0.2,    # 特征1
  #     有毛发:0.6,    # 特征2
  #     会飞:0.7,      # 特征3
  #     在水里:0.1,    # 特征4
  #     ...            # 共64个特征
  # ]
  #
  # # 每个类别有自己的"期望特征":
  # 飞机模板 = [
  #     翅膀权重:+2.0,   # 很重要!
  #     轮子权重:-0.5,   # 不要有轮子
  #     毛发权重:-1.0,   # 不要有毛发
  #     ...
  # ]
  #
  # 汽车模板 = [
  #     翅膀权重:-1.0,   # 不要有翅膀
  #     轮子权重:+3.0,   # 很重要!
  #     毛发权重:-0.5,
  #     ...
  # ]
  #
  # # 计算"匹配度":
  # logit[飞机] = 0.8×(+2.0) + 0.2×(-0.5) + 0.6×(-1.0) + ...
  #             = 1.6 - 0.1 - 0.6 + ...
  #             = 高分!因为"有翅膀"特征强
  #
  # logit[汽车] = 0.8×(-1.0) + 0.2×(+3.0) + 0.6×(-0.5) + ...
  #             = -0.8 + 0.6 - 0.3 + ...
  #             = 低分,因为"轮子"特征弱
  #
  # 4. 为什么叫"未归一化"?
  #
  # # Logits 的特点:
  # logits = [1.88, 0.27, 0.51, -0.90, -0.74, ...]
  #          ↑     ↑     ↑      ↑       ↑
  #         飞机  汽车   鸟     猫      鹿
  #
  # # 注意:
  # # 1. 可以是负数 ❌ (不像概率必须≥0)
  # # 2. 和不是1   ❌ (不像概率和必须=1)
  # # 3. 可以很大   ❌ (不像概率≤1)
  #
  # # 这就是"未归一化"的意思!
  #
  # # 通过 softmax 归一化:
  # probs = softmax(logits) = [0.447, 0.089, 0.113, 0.028, 0.032, ...]
  #                            ↑     ↑     ↑      ↑      ↑
  #                           ✅    ✅    ✅     ✅     ✅
  #                          全是正数,和=1,都≤1
  #
  # 5. 核心公式总结
  #
  # # 完整流程的数学表达:
  #
  # # 步骤1: 特征提取
  # features = CNN(image)  # (batch, 64, 4, 4)
  # features = flatten(features)  # (batch, 1024)
  #
  # # 步骤2: 降维
  # semantic = ReLU(W₁ @ features + b₁)  # (batch, 64)
  #
  # # 步骤3: 分类(生成logits)
  # logits = W₂ @ semantic + b₂  # (batch, 10)
  # #        ↑
  # #    关键!这一步把64维特征映射到10类打分
  #
  # # 步骤4: 计算概率(softmax)
  # probs = exp(logits) / Σ(exp(logits))  # (batch, 10)
  #
  # # 步骤5: 计算损失(交叉熵)
  # loss = -log(probs[正确类别])
  #
  # 🎓 最终理解
  #
  # Logits 的本质:
  #
  # 1. 物理意义: 特征向量与各类别"模板"的相似度评分
  # 2. 数学形式: 线性变换的输出,未经非线性归一化
  # 3. 数值特点: 无界的实数,可正可负,可大可小
  # 4. 作用:
  #   - 训练时:送入交叉熵计算损失(内部会softmax)
  #   - 推理时:argmax找最大值即为预测类别
  #
  # 为什么这样设计?
  #
  # # ✅ 好处:
  # # 1. 数值稳定:直接计算log(softmax)比先softmax再log稳定
  # # 2. 梯度友好:线性输出的梯度更容易传播
  # # 3. 灵活性:可以用不同方式解释(概率/能量/距离)
  #
  # # 记忆:
  # # Logits = "逻辑回归的输出" (Logistic regression units)
  # #        = 决策边界上的"原始得分"
  #        = 还没转成概率的"评分"